## Setup

In [1]:
%config Completer.use_jedi = False

import os

# selenium for downloading amazon reports
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys

from amazoncaptcha import AmazonCaptcha

import subprocess  # for shell commands to Onepassword
from getpass import getpass
import json
import pandas as pd
import numpy as np
import pathlib
import time
import datetime
from distutils import spawn

import pandas as pd
import numpy as np
import json
import os
import requests
from pprint import pprint
from time import strftime, strptime
from datetime import datetime, timedelta

In [3]:
# make sure program directory has a place to download csvs, 
# then step into that dir
DOWNLOAD_DIR = os.path.join(os.getcwd(), "download")
if not os.path.exists(DOWNLOAD_DIR):
    os.makedirs(DOWNLOAD_DIR)

os.chdir(DOWNLOAD_DIR)

## Function Definitions

In [346]:
# 1Password functions: -----------------------------
def get_op_session_token(pw):
    """Generate OnePassword Session Token

    Args:
        pw (str): your onepassword master password

    Returns:
        str: your onepassword session token; works for 30 minutes.
    """
    pw_b = pw.encode('utf-8')
    cmd = "op signin my --raw".split()
    
    out = subprocess.run(cmd, input=pw_b, capture_output=True)
    if out.stdout:
        op_token = out.stdout.decode('utf-8').rstrip()
    else:
        err = out.stderr.decode('utf-8').rstrip()
        raise Exception(err)

    return op_token


def get_op_totp(item, session_token):
    cmd = f"op get totp {item} --session {session_token}".split()
    return subprocess.run(cmd, capture_output=True, text=True).stdout.rstrip()


def get_op_field(item, field, session_token):
    cmd = (
        f"op get item {item} --fields {field} --session {session_token}"
        .split()
    )
    return subprocess.run(cmd, capture_output=True, text=True).stdout.rstrip()


# Selenium functions ------------------------------------------------
def download_amz_report(which_report, op_token, download_dir=os.getcwd()):
    """
    Use Selenium to download an Amazon B2B report from 
    https://www.amazon.com/gp/b2b/reports. Saves to CSV.

    Args:
        which_report (str): either "items" or "orders"
        download_dir (str, optional): path to download directory. 
                                      Defaults to os.getcwd().

    Returns:
        None
    """
    assert which_report in ["items", "orders"]

    # set up options for where to download the report
    chrome_options = webdriver.ChromeOptions()
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--verbose")
    prefs = { "download.default_directory" : download_dir }
    chrome_options.add_experimental_option("prefs", prefs)

    # initiate the webdriver
    chromedriver = spawn.find_executable("chromedriver")
    driver = webdriver.Chrome(chromedriver, options=chrome_options)

    driver.get("https://www.amazon.com/gp/b2b/reports")
    driver.find_element_by_id("ap_email").send_keys(get_op_field("Amazon", "username", op_token))
    driver.find_element_by_id("continue").click()
    time.sleep(np.random.random() * 10)
    driver.find_element_by_id("ap_password").send_keys(get_op_field("Amazon", "password", op_token))
    driver.find_element_by_id("signInSubmit").click()
    time.sleep(np.random.random() * 10)
    driver.find_element_by_id("auth-mfa-otpcode").send_keys(get_op_totp("Amazon", op_token))
    driver.find_element_by_id("auth-signin-button").click()
    time.sleep(np.random.random() * 10)
    driver.find_element_by_id("report-type").send_keys([which_report, Keys.RETURN])
    driver.find_element_by_id("report-last30Days").click()
    driver.find_element_by_id("report-type").click()
    driver.find_element_by_id("report-confirm").click()

    time.sleep(np.random.random() * 10)

    def get_latest_csv():
        files = [f for f in os.listdir() if f.endswith(".csv") or f.endswith(".CSV")]
        return max(files, key=os.path.getctime)

    file = get_latest_csv()
    os.rename(file, f"{which_report}.csv")
    print(f"Renamed {file} to {which_report}.csv")
    driver.quit()



def load_and_process_amz_csv(csv):
    df = pd.read_csv(csv)
    r, c = df.shape
    print(f"Loaded {r} rows and {c} columns from {csv}.")
    
    # cast all colnames to lowercase and replace spaces with underscores
    df.columns = [c.lower().replace(" ", "_") for c in df.columns]

    # identify money related columns based on keywords
    keywords = ["total", "charge", "amount"]
    moneycols = df.columns[
        df.columns.str.contains("|".join(keywords))
    ]

    # format money columns like YNAB: $10.00 charge = -1000
    def reformat_moneycols_like_ynab(c):
        return (
            c.str.replace("$", "", regex=False)
            .astype(float)
            .multiply(-1000)
            .astype(int)
        )

    df[moneycols] = (
        df[moneycols].apply(reformat_moneycols_like_ynab, axis=1)
    )
    print(f"Converted columns {', '.join(moneycols)} to YNAB currency format.")

    # reformat date columns to datetime:
    datecols = df.columns[df.columns.str.contains("date")]
    df[datecols] = df[datecols].apply(lambda x: pd.to_datetime(x))
    print(f"Converted columns {', '.join(datecols)} to pandas datetime format.")

    return df



# YNAB FUNCTIONS ==================================================
def get_ynab_transactions(budget_id, token, since_date=None):
    base_url = "https://api.youneedabudget.com/v1"
    request_url = f"{base_url}/budgets/{budget_id}/transactions"
    r = requests.get(request_url, params = {"access_token": token, "since_date": since_date})
    if since_date:
        print(f"Downloading transactions from YNAB API since {since_date}...")
    else:
        print(f"Downloading all transactions from YNAB API...")

    print(r)
    j = json.loads(r.text)
    ynab_tx = pd.json_normalize(j['data']['transactions'])
    return ynab_tx


def upload_tx_to_ynab(ynab_tx, budget_id, token):
    base_url = "https://api.youneedabudget.com/v1"
    request_url = f"{base_url}/budgets/{budget_id}/transactions"

    ynab_tx_js = ynab_tx.to_json(orient='records')
    ynab_j = json.loads(ynab_tx_js)

    j = json.dumps({
        "transactions": ynab_j
    })

    js = json.loads(j)

    r = requests.patch(
        request_url, 
        data=j, 
        headers={
            "Authorization": f"Bearer {token}", 
            'Content-Type':'application/json'
        }
    )

    print("YNAB API Response:")
    print(r)

## Log into OP and Get Creds

In [171]:
op_token = get_op_session_token(pw=getpass("Enter OP Master Password: "))

Enter OP Master Password: ········


## Get Reports with Selenium

In [472]:
# download the items and orders csvs -- only if necessary
if not os.path.exists('items.csv'):
    print("Getting Amazon Items Report... This might take a bit...")
    download_amz_report(which_report="items", op_token=op_token)
else:
    print("Items CSV exists, skipping download")
if not os.path.exists('orders.csv'):
    print("Getting Amazon Orders Report... This might take a bit...")
    download_amz_report(which_report="orders", op_token=op_token)
else:
    print("Orders CSV exists, skipping download")

orders = (
    load_and_process_amz_csv('orders.csv')
    .groupby('order_id')
    .agg({"total_charged": sum})
    .reset_index()
)

items = load_and_process_amz_csv('items.csv')

Items CSV exists, skipping download
Orders CSV exists, skipping download
Loaded 8 rows and 23 columns from orders.csv.
Converted columns subtotal, shipping_charge, total_promotions, tax_charged, total_charged to YNAB currency format.
Converted columns order_date, shipment_date to pandas datetime format.
Loaded 11 rows and 36 columns from items.csv.
Converted columns item_subtotal, item_subtotal_tax, item_total to YNAB currency format.
Converted columns order_date, release_date, shipment_date to pandas datetime format.


## Group Amazon Transactions by Order ID
This section groups the transactions found in the items report and order report by order id, such that each record contains the individual item information as well as its corresponding order membership. 

Then it performs some aggregating, getting ultimately each order group's total amount, an appropriate memo, and, where applicable, arrays of subtransactions.

In [483]:
amz_merged = pd.merge(items, orders)
amz_merged['subtransactions'] = (
    amz_merged.apply(lambda x: {"amount": x.item_total,
                                "memo": x.title}, axis=1)
)

amz_merged = amz_merged.rename(columns={"title": "memo"})

amz_merged['item_count'] = 1

amz_merged = (
    amz_merged
    .groupby('order_id')
    .agg({
        "total_charged": np.mean
        , "subtransactions": list
        , "item_count": sum
        , "memo": list
    })
    .reset_index(drop=True)
)


amz_merged.subtransactions = (
    amz_merged
    .apply(lambda x: [] if x.item_count == 1 else x.subtransactions, 
           axis=1)
)

amz_merged['memo'] = (
    amz_merged.apply(lambda x: f"({x.item_count} items)" 
                     if x.item_count > 1 
                     else x.memo[0],
                     axis=1)
)


amz_merged = amz_merged[['total_charged', 'memo', 'subtransactions']]

## Get YNAB Transactions

In [475]:
op_token = get_op_session_token(pw=getpass("Enter OP Master Password: "))

YNAB_KEY = get_op_field(item='YNAB', field="API_TOKEN", session_token=op_token)
YNAB_BUDGET_ID = get_op_field(item='YNAB', field='BUDGET_ID', session_token=op_token)

# set a date 7 days before the earliest transaction in the amazon
# reports to make sure we get all the relevant YNAB transactions
ynab_since_date = (
    ( min(orders_memoized.order_date) - timedelta(days=7) )
    .strftime("%Y-%m-%d")
)

# YNAB transactions -------------------------------------------------------
ynab_tx = get_ynab_transactions(
    budget_id=YNAB_BUDGET_ID,
    since_date=ynab_since_date,
    token=YNAB_KEY
)

Enter OP Master Password: ········
<Response [200]>


In [480]:
ynab_tx_amz_new = (
    ynab_tx
    .query("account_name.str.contains('AMZ')")
    .query("category_name == 'Uncategorized'")
    .reset_index(drop=True)
)

## Populate YNAB Records with Subtransactions

In [481]:
ynab_matched_tx = (
    pd.merge(ynab_tx_amz_new,
             amz_merged, 
             how='left',
             left_on='amount',
             right_on='total_charged')
)

ynab_matched_tx = (
    ynab_matched_tx
    .rename(columns={"memo_y": "memo", 
                     "subtransactions_y": "subtransactions"})
    [['id', 'memo', 'subtransactions', 'flag_color']]
)

ynab_matched_tx.memo = (
    ynab_matched_tx.memo.replace(np.nan, np.nan, regex=True)
)

ynab_matched_tx.subtransactions = (
    ynab_matched_tx.subtransactions.apply(lambda x: [] 
                                          if x is np.nan 
                                          else x)
)

ynab_matched_tx.flag_color = (
    ynab_matched_tx.apply(lambda x: "red" 
                          if x.memo is np.nan 
                          else None, axis=1)
)


## Upload the New Transaction Memos to YNAB

The below uploads just the {transaction id: memo} pairs back to YNAB via API PATCH method.

In [479]:
upload_tx_to_ynab(ynab_tx=ynab_matched_tx[['id', 'memo', 'subtransactions', 'flag_color']], 
                  budget_id=YNAB_BUDGET_ID,
                  token=YNAB_KEY)

YNAB API Response:
<Response [200]>


In [131]:
def upload_tx_to_ynab(ynab_tx, budget_id, token):
    base_url = "https://api.youneedabudget.com/v1"
    request_url = f"{base_url}/budgets/{budget_id}/transactions"

    ynab_tx_js = ynab_tx.to_json(orient='records')
    ynab_j = json.loads(ynab_tx_js)

    j = json.dumps({
        "transactions": ynab_j
    })

    js = json.loads(j)

    r = requests.patch(
        request_url, 
        data=j, 
        headers={
            "Authorization": f"Bearer {token}", 
            'Content-Type':'application/json'
        }
    )

    print("YNAB API Response:")
    print(r)